In [ ]:
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from ml_wireless_classification.base.SignalUtils import (
    compute_instantaneous_features, compute_modulation_index, compute_spectral_asymmetry,
    instantaneous_frequency_deviation, spectral_entropy, envelope_mean_variance,
    spectral_flatness, spectral_peaks_bandwidth, zero_crossing_rate, compute_fft_features,
    autocorrelation, is_digital_signal, compute_kurtosis, compute_skewness,
    compute_spectral_energy_concentration, compute_instantaneous_frequency_jitter,
    compute_spectral_kurtosis, compute_higher_order_cumulants, compute_crest_factor,
    compute_spectral_entropy, compute_energy_spread, compute_autocorrelation_decay,
    compute_rms_of_instantaneous_frequency, compute_entropy_of_instantaneous_frequency,
    compute_envelope_variance, compute_papr
)
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel
# Global dictionary to store feature names and values
feature_dict = {}

def add_feature(name, func, *args):
    """Try to add a feature by checking the shape and ensuring it’s a scalar."""
    try:
        value = func(*args)
        if np.isscalar(value):
            feature_dict[name] = value
        elif isinstance(value, (list, tuple, np.ndarray)) and value.size == 1:
            feature_dict[name] = value.item()
        else:
            print(f"Warning: Feature '{name}' has incorrect shape and was not added.")
    except Exception as e:
        print(f"Error computing feature '{name}': {e}")
        

def compute_kernel_transformed_features(signal, kernel_type='rbf', gamma=0.1, degree=2):
    """
    Compute kernel-based features to help distinguish QAM levels.
    
    Parameters:
    - signal: The input complex signal.
    - kernel_type: Type of kernel to use ('rbf' or 'polynomial').
    - gamma: Parameter for the RBF kernel.
    - degree: Degree for the polynomial kernel.
    
    Returns:
    - A scalar feature based on the chosen kernel.
    """
    magnitudes = np.abs(signal)
    distances = np.expand_dims(magnitudes, axis=1)  # Reshape for pairwise kernel computation
    
    if kernel_type == 'rbf':
        kernel_matrix = rbf_kernel(distances, gamma=gamma)
    elif kernel_type == 'polynomial':
        kernel_matrix = polynomial_kernel(distances, degree=degree, gamma=gamma)
    else:
        raise ValueError("Invalid kernel type. Choose 'rbf' or 'polynomial'.")
    
    # Summarize kernel matrix into a scalar feature (e.g., mean or variance)
    mean_kernel_value = np.mean(kernel_matrix)
    variance_kernel_value = np.var(kernel_matrix)
    
    return mean_kernel_value, variance_kernel_value

def extract_features(data):
    features = []
    labels = []
    snrs = []
    # Kernel parameters for RBF and polynomial kernels
    rbf_gammas = [0.5, 0.6, 0.7, 0.8, 0.9]
    poly_degrees = [3, 4, 5]
    
    for key, signals in data.items():
        mod_type, snr = key
        for signal in signals:
            real_part, imag_part = signal[0], signal[1]
            complex_signal = real_part + 1j * imag_part

            # Reset feature dictionary for each signal
            global feature_dict
            feature_dict = {}

            # # Add features with validation
            add_feature("Inst. Freq. Dev", instantaneous_frequency_deviation, complex_signal)
            add_feature("Phase Variance", lambda x: np.var(compute_instantaneous_features(x)[1]), real_part)
            # # Additional features for QAM16 vs QAM64 separation
            add_feature("Avg Symbol Power", lambda x: np.mean(np.abs(x)**2), complex_signal)
            add_feature("PAPR", lambda x: np.max(np.abs(x)**2) / np.mean(np.abs(x)**2), complex_signal)
            add_feature("Kurtosis Magnitude", lambda x: compute_kurtosis(np.abs(x)), complex_signal)
            add_feature("Skewness Magnitude", lambda x: compute_skewness(np.abs(x)), complex_signal)

            # Add kernel-based features for different RBF gamma values
            for gamma in rbf_gammas:
                mean_rbf, var_rbf = compute_kernel_transformed_features(complex_signal, kernel_type='rbf', gamma=gamma)
                add_feature(f"RBF Kernel Mean Gamma_{gamma}", lambda: mean_rbf)
                add_feature(f"RBF Kernel Variance Gamma_{gamma}", lambda: var_rbf)
            
            # Add kernel-based features for different polynomial degrees
            for degree in poly_degrees:
                mean_poly, var_poly = compute_kernel_transformed_features(complex_signal, kernel_type='polynomial', gamma=0.1, degree=degree)
                add_feature(f"Poly Kernel Mean Degree_{degree}", lambda: mean_poly)
                add_feature(f"Poly Kernel Variance Degree_{degree}", lambda: var_poly)


            # Add SNR as a feature
            feature_dict["SNR"] = snr  # Include SNR as part of the features

            # Append the feature values and label
            features.append(list(feature_dict.values()))
            labels.append(mod_type)

    return np.array(features), labels

# Load the RML2016.10a_dict.pkl file with explicit encoding
with open("../RML2016.10a_dict.pkl", "rb") as f:
    data = pickle.load(f, encoding="latin1")

# Feature extraction for all signals
features, labels = extract_features(data)


In [ ]:
# Encode labels for classification
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.3, random_state=42)

# Train a single classifier on the entire dataset
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)


In [ ]:
# Evaluate accuracy for each SNR level
unique_snrs = sorted(set(X_test[:, -1]))  # Get unique SNR levels from test set
accuracy_per_snr = []

for snr in unique_snrs:
    # Select samples with the current SNR
    snr_indices = np.where(X_test[:, -1] == snr)
    X_snr = X_test[snr_indices]
    y_snr = y_test[snr_indices]

    # Predict and calculate accuracy
    y_pred = clf.predict(X_snr)
    accuracy = accuracy_score(y_snr, y_pred)
    accuracy_per_snr.append(accuracy * 100)  # Convert to percentage

    print(f"SNR: {snr} dB, Accuracy: {accuracy * 100:.2f}%")

# Plot Recognition Accuracy vs. SNR
plt.figure(figsize=(10, 6))
plt.plot(unique_snrs, accuracy_per_snr, 'b-o', label='Recognition Accuracy')
plt.xlabel("SNR (dB)")
plt.ylabel("Recognition Accuracy (%)")
plt.title("Recognition Accuracy vs. SNR")
plt.legend()
plt.grid(True)
plt.ylim(0, 100)
plt.show()

In [ ]:

# Feature importance for the classifier
feature_names = list(feature_dict.keys())
importances = clf.feature_importances_

# Sort feature importances in descending order
sorted_indices = np.argsort(importances)[::-1]
sorted_feature_names = [feature_names[i] for i in sorted_indices]
sorted_importances = importances[sorted_indices]

# Plot sorted feature importances
plt.figure(figsize=(10, 8))
plt.barh(sorted_feature_names, sorted_importances, color='skyblue')
plt.xlabel("Feature Importance")
plt.title("Feature Importance for Modulation Classification")
plt.gca().invert_yaxis()  # Invert y-axis to show the highest importance at the top
plt.show()

# Confusion matrix for overall test set
y_pred_test = clf.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for Multi-Class Modulation Classification")
plt.show()

# Print Classification Report
print("Classification Report for Modulation Types:")
print(classification_report(y_test, y_pred_test, target_names=label_encoder.classes_))

In [ ]:

# Evaluate accuracy for each SNR level
unique_snrs = sorted(set(X_test[:, -1]))  # Get unique SNR levels from test set
accuracy_per_snr = []

for snr in unique_snrs:
    # Select samples with the current SNR
    snr_indices = np.where(X_test[:, -1] == snr)
    X_snr = X_test[snr_indices]
    y_snr = y_test[snr_indices]

    # Predict and calculate accuracy
    y_pred = clf.predict(X_snr)
    accuracy = accuracy_score(y_snr, y_pred)
    accuracy_per_snr.append(accuracy * 100)  # Convert to percentage

    print(f"SNR: {snr} dB, Accuracy: {accuracy * 100:.2f}%")

# Plot Recognition Accuracy vs. SNR
plt.figure(figsize=(10, 6))
plt.plot(unique_snrs, accuracy_per_snr, 'b-o', label='Recognition Accuracy')
plt.xlabel("SNR (dB)")
plt.ylabel("Recognition Accuracy (%)")
plt.title("Recognition Accuracy vs. SNR for Modulation Classification")
plt.legend()
plt.grid(True)
plt.ylim(0, 100)
plt.show()

# Feature importance for the classifier
feature_names = list(feature_dict.keys())
importances = clf.feature_importances_
plt.figure(figsize=(10, 8))
plt.barh(feature_names, importances, color='skyblue')
plt.xlabel("Feature Importance")
plt.title("Feature Importance for Modulation Classification")
plt.show()

# Confusion matrix for overall test set
y_pred_test = clf.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for Multi-Class Modulation Classification")
plt.show()

# Print Classification Report
print("Classification Report for Modulation Types:")
print(classification_report(y_test, y_pred_test, target_names=label_encoder.classes_))
